## Metric Calculation:
1. Safely calculating Loss Ratio by province.
2. Aggregating data for Claim Frequency and Average Claim Amount by month.
3.  Preparing data for Vehicle Make Risk Profile.
### Professional Visualizations: Generating three key plots:
1. Loss Ratio by Province: With profitability markers.
2.  Temporal Trends in Claims: Dual-axis plot for frequency and severity.
3. Vehicle Make Risk Profile: Claim frequency and distribution of claim amounts by make.

In [1]:
import sys
import pandas as pd

from src import load_clean_data, load_raw_data, plot_distributions,save_processed_data,detect_outliers, create_insightful_visuals, calculate_aggregated_metrics,  analyze_and_report_ab_test

sys.path.append("../src")
file_path  = '../data/raw/MachineLearningRating_v3.txt'
insurance_df = load_raw_data(file_path)
insurance_df.head()

INFO:src.utils.load_data:Loaded 1000098 records from ../data/raw/MachineLearningRating_v3.txt.


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,9.000000e-13
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.000000e+00
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.000000e+00
3,1452155,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.000000e+00
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.000000e+00


In [2]:


vehicle_type_mode = insurance_df['VehicleType'].mode()[0]
insurance_df['VehicleType'] = insurance_df['VehicleType'].fillna(vehicle_type_mode)
insurance_df['CustomValueEstimate'] = insurance_df.groupby('VehicleType')['CustomValueEstimate'].transform(lambda x: x.fillna(x.median()))
# Impute categorical columns
for col in ['Gender', 'MaritalStatus', 'Bank', 'AccountType', 'WrittenOff', 'Rebuilt', 'Converted']:
    insurance_df[col] = insurance_df[col].fillna(insurance_df[col].mode()[0])
for col in ['mmcode', 'VehicleType', 'make', 'Model', 'bodytype', 'VehicleIntroDate']:
    insurance_df[col] = insurance_df[col].fillna(insurance_df[col].mode()[0])
# Impute numerical columns
for col in ['Cylinders', 'cubiccapacity', 'kilowatts', 'NumberOfDoors', 'CapitalOutstanding']:
    insurance_df[col] = pd.to_numeric(insurance_df[col], errors='coerce')
    insurance_df[col] = insurance_df[col].fillna(int(insurance_df[col].median()))
    insurance_df['NewVehicle'] = insurance_df['NewVehicle'].fillna(
    insurance_df['RegistrationYear'].apply(lambda x: 'Yes' if x >= 2023 else 'No'))

insurance_df = insurance_df.drop('NumberOfVehiclesInFleet', axis=1)
insurance_df = insurance_df.drop('CrossBorder', axis=1)


    # Handle missing values
# numerical_cols = insurance_df.select_dtypes(include=['int64', 'float64']).columns
# categorical_cols = insurance_df.select_dtypes(include=['object']).columns
#
# for col in numerical_cols:
#     insurance_df[col] = insurance_df[col].fillna(insurance_df[col].median())
#
# for col in categorical_cols:
#     insurance_df[col] = insurance_df[col].fillna(insurance_df[col].mode()[0])
insurance_df.isnull().sum()

UnderwrittenCoverID         0
PolicyID                    0
TransactionMonth            0
IsVATRegistered             0
Citizenship                 0
LegalType                   0
Title                       0
Language                    0
Bank                        0
AccountType                 0
MaritalStatus               0
Gender                      0
Country                     0
Province                    0
PostalCode                  0
MainCrestaZone              0
SubCrestaZone               0
ItemType                    0
mmcode                      0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity               0
kilowatts                   0
bodytype                    0
NumberOfDoors               0
VehicleIntroDate            0
CustomValueEstimate         0
AlarmImmobiliser            0
TrackingDevice              0
CapitalOutstanding          0
NewVehicle

### calculate aggregated metrics

In [3]:
aggregated_df = calculate_aggregated_metrics(insurance_df)
aggregated_df.isnull().sum()

Province             0
PostalCode           0
Gender               0
ClaimFrequency       0
TotalPolicies        0
ClaimCount           0
TotalPremiums        0
TotalClaimAmounts    0
ClaimSeverity        0
Margin               0
dtype: int64

In [4]:
print(aggregated_df.tail())

         Province  PostalCode         Gender  ClaimFrequency  TotalPolicies  \
978  Western Cape        8020           Male        0.001792            558   
979  Western Cape        8020  Not specified        0.001628           4299   
980  Western Cape        8040  Not specified        0.000000            390   
981  Western Cape        8051  Not specified        0.003386            886   
982  Western Cape        8060  Not specified        0.003643            549   

     ClaimCount  TotalPremiums  TotalClaimAmounts  ClaimSeverity        Margin  
978           1   13606.915690        4258.263158    4258.263158   9348.652533  
979           7  301095.673685      303430.939298   43347.277043  -2335.265613  
980           0   34186.621619           0.000000       0.000000  34186.621619  
981           3   31717.301055       47488.912281   15829.637427 -15771.611225  
982           2   46676.236996       45745.614035   22872.807018    930.622961  


In [5]:
import  sys
sys.path.append("../src")
from src import  analyze_and_report_ab_test

 # Step 3: Analyze and report using A/B testing
analyze_and_report_ab_test(aggregated_df)

--- A/B Hypothesis Testing Analysis Report ---
Significance Level (alpha): 0.05

## 1. Province Risk and Profit Differences
H₀: No risk difference (Claim Frequency) across provinces (Eastern Cape vs Free State): **FAILED TO REJECT** (p = 0.6705).
   Interpretation: No statistically significant difference in Claim Frequency was found between Province Eastern Cape and Province Free State.
   Mean Freq Eastern Cape: 0.0013, Mean Freq Free State: 0.0016
H₀: No risk difference (Claim Severity) across provinces (Eastern Cape vs Free State): **FAILED TO REJECT** (p = 0.7799).
   Interpretation: No statistically significant difference in Claim Severity was found between Province Eastern Cape and Province Free State.
   Mean Severity Eastern Cape: 8748.96, Mean Severity Free State: 10703.47
H₀: No margin difference across provinces (Eastern Cape vs Free State): **FAILED TO REJECT** (p = 0.7027).
   Interpretation: No statistically significant difference in Margin was found between Province East